# Performance of Multi-zone VAV System under Summer Part-Load Conditions

We look at the VAV system from example_05 under the following summer part-load conditions:  

<div class="my_table">
    <table>
        <tr>
            <th></th>
            <th>Zone A (Exterior)</th>
            <th>Zone B (Interior)</th>
        </tr>
        <tr>
            <td>Sensible cooling load</td>
            <td>143,100 Btu/hr</td>
            <td>91,260 Btu/hr</td>
        </tr>
        <tr>
            <td>Latent cooling load</td>
            <td>36,000 Btu/hr</td>
            <td>20,000 Btu/hr</td>
        </tr>
        <tr>
            <td>Zone temperature</td>
            <td>75°F</td>
            <td>75°F</td>
        </tr>
    </table>
</div>

In [1]:
from hvac import Quantity
from hvac.fluids import HumidAir
from hvac.air_conditioning.multi_zone import Zone, Season
from hvac.air_conditioning.multi_zone import part_load

In [2]:
Q_ = Quantity

## Zone A

- Specify the sensible and latent part load of the zone.
- Specify the desired state of zone air.
- Specify the full-load mass flow rate of supply air to the zone determined when designing the system.

In [3]:
zone_A = Zone(
    name='zone A',
    summer=Season(
        Q_sen=Q_(143_100, 'Btu / hr'),
        Q_lat=Q_(36_000, 'Btu / hr'),
        zone_air=HumidAir(
            Tdb=Q_(75, 'degF'),
            RH=Q_(50, 'pct')
        ),
        m_supply=Q_(769.1, 'lb / min')
    )
)

## Zone B

In [4]:
zone_B = Zone(
    name='zone B',
    summer=Season(
        Q_sen=Q_(91_260, 'Btu / hr'),
        Q_lat=Q_(20_000, 'Btu / hr'),
        zone_air=HumidAir(
            Tdb=Q_(75, 'degF'),
            RH=Q_(50, 'pct')
        ),
        m_supply=Q_(353.4, 'lb / min')
    )
)

## Define the VAV-System

- Add zones to the VAV system
- Specify the system supply air temperature determined when designing the system.
- Specify the outdoor air state at part load
- Specify the ventilation volume flow rate

In [5]:
vav_system = part_load.VAVSystem(
    zones=[zone_A, zone_B],
    T_supply=Q_(55, 'degF'),
    outdoor_air=HumidAir(
        Tdb=Q_(77, 'degF'),
        W=Q_(0.0126, 'lb / lb')
    ),
    V_vent=Q_(2400, 'ft ** 3 / min')
)

## Run Part-Load Analysis

In [6]:
results = vav_system.part_load_summer(
    dP_fan_sup=Q_(3, 'inch_H2O_60F'),
    eta_fan_sup=Q_(60, 'pct'),
    Q_duct_sup=None,
    dP_fan_ret=None,
    eta_fan_ret=None,
    Q_duct_ret=None
)

### Cooling Coil Load

In [7]:
print(
    "sensible cooling coil load = "
    f"{results['Q_cc_sen'].to('Btu / hr'):~P.0f}",
    "latent cooling coil load = "
    f"{results['Q_cc_lat'].to('Btu / hr'):~P.0f}",
    "total cooling coil load = "
    f"{results['Q_cc_tot'].to('Btu / hr'):~P.0f}",
    sep='\n'
)

sensible cooling coil load = -260521 Btu/h
latent cooling coil load = -122063 Btu/h
total cooling coil load = -382210 Btu/h


### Reheat Coil Loads

**Zone A**

In [8]:
print(
    "reheat coil load = "
    f"{zone_A.reheat_coil.Q_sen.to('Btu / hr'):~P.0f}"
)

reheat coil load = -0 Btu/h


**Zone B**

In [9]:
print(
    "reheat coil load = "
    f"{zone_B.reheat_coil.Q_sen.to('Btu / hr'):~P.0f}"
)

reheat coil load = -0 Btu/h


**Total reheat coil load**

In [10]:
print(
    "total reheat coil load = "
    f"{results['Q_rh_tot'].to('Btu / hr'):~P.0f}"
)

total reheat coil load = -0 Btu/h


### Supply Air Flow Rates

**Zone A**

In [11]:
print(
    "supply air mass flow rate zone A = "
    f"{zone_A.summer.m_supply.to('lb / min'):~P.1f}",
    "supply air volume flow rate zone A = "
    f"{zone_A.summer.V_supply.to('ft ** 3 / min'):~P.1f}",
    sep='\n'
)

supply air mass flow rate zone A = 489.5 lb/min
supply air volume flow rate zone A = 6431.3 ft³/min


**Zone B**

In [12]:
print(
    "supply air mass flow rate zone B = "
    f"{zone_B.summer.m_supply.to('lb / min'):~P.1f}",
    "supply air volume flow rate zone B = "
    f"{zone_B.summer.V_supply.to('ft ** 3 / min'):~P.1f}",
    sep='\n'
)

supply air mass flow rate zone B = 312.2 lb/min
supply air volume flow rate zone B = 4102.4 ft³/min


**System**

In [13]:
print(
    "system supply air mass flow rate = "
    f"{vav_system.summer.m_supply.to('lb / min'):~P.1f}",
    "system supply air volume flow rate zone B = "
    f"{vav_system.summer.V_supply.to('ft ** 3 / min'):~P.1f}",
    sep='\n'
)

system supply air mass flow rate = 801.6 lb/min
system supply air volume flow rate zone B = 10533.7 ft³/min


### Zone Air States

**Zone A**

In [14]:
print(
    "resulting state of air in zone A = "
    f"{zone_A.summer.return_air.Tdb.to('degF'):~P.0f} DB, "
    f"{zone_A.summer.return_air.RH.to('pct'):~P.0f} RH"
)

resulting state of air in zone A = 75 °F DB, 50 % RH


**Zone B**

In [15]:
print(
    "resulting state of air in zone B = "
    f"{zone_B.summer.return_air.Tdb.to('degF'):~P.0f} DB, "
    f"{zone_B.summer.return_air.RH.to('pct'):~P.0f} RH"
)

resulting state of air in zone B = 75 °F DB, 50 % RH


### Entering and Leaving Air State at Cooling Coil

In [16]:
print(
    "Entering air state at cooling coil = "
    f"{vav_system.summer.mixed_air.Tdb.to('degF'):~P.0f} DB, "
    f"{vav_system.summer.mixed_air.RH.to('pct'):~P.0f} RH, "
    f"{vav_system.summer.mixed_air.W.to('lb / lb'):~P.4f} lb_w / lb_a"
)

Entering air state at cooling coil = 75 °F DB, 53 % RH, 0.0100 lb_w / lb_a


In [17]:
print(
    "Leaving air state at cooling coil = "
    f"{vav_system.summer.cooled_air.Tdb.to('degF'):~P.0f} DB, "
    f"{vav_system.summer.cooled_air.RH.to('pct'):~P.0f} RH, "
    f"{vav_system.summer.cooled_air.W.to('lb / lb'):~P.4f} lb_w / lb_a"
)

Leaving air state at cooling coil = 53 °F DB, 89 % RH, 0.0077 lb_w / lb_a
